In [1]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme("chesterish")

In [2]:
import numpy as np
import pygame as p
import sys

pygame 2.0.0 (SDL 2.0.12, python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Constants:
WIDTH, HEIGHT = 400, 400
ROWS, COLS, = 8, 8
BOX_SIZE = WIDTH//COLS
RED, GREEN, BLUE = (255,0,0), (0,255,0), (0,0,255)
WHITE, BLACK, SILVER = (255, 255, 255), (0,0,0), (192,192,192)
SALMON = (252,151,151)

image_dimensions = 17
CROWN = p.transform.scale(p.image.load("crown.png"), (image_dimensions,image_dimensions))

In [4]:
DARK =  (184,139,74)
LIGHT = (227,193,111)

In [5]:
class Piece:

  PADDING = 14
  BORDER = 2

  def __init__(self, row, column, colour):
    self.row = row
    self.column = column
    self.colour = colour
    if self.colour == WHITE:
      self.direction = 1
    else:
      self.direction = -1
    self.king = False
    self.x = 0
    self.y = 0
    self.calculate_position()

  def set_king(self):
    self.king = True

  def calculate_position(self):
    self.x = BOX_SIZE * self.column + BOX_SIZE // 2
    self.y = BOX_SIZE * self.row + BOX_SIZE // 2

  def draw(self, win):
    radius = BOX_SIZE//2 - self.PADDING
    p.draw.circle(win, SILVER, (self.x,self.y), radius + self.BORDER)
    p.draw.circle(win, self.colour, (self.x,self.y), radius)
    if self.king:
      new_x = self.x - CROWN.get_width()//2
      new_y = self.y - CROWN.get_height()//2
      win.blit(CROWN, (new_x, new_y))

  def move_piece(self, row, column):
    self.row = row
    self.column = column
    self.calculate_position()

  def __repr__(self):
    if self.direction > 0:
      return "+"
    else:
      return "-"

In [34]:
class GameBoard(object):


  def __init__(self):
    self.board = np.zeros(shape=(8,8)).astype("int").tolist()
    self.player_count = 0
    self.red_remaining = self.white_remaining = 12
    self.red_king_count = self.white_king_count = 0
    self.setup_board()


  def setup_board(self):
    # Start player 2 at "top" of board.
    colour = 2
    # Counter to switch to using player 1 identifier.
    counter = 0 
    ln = len(self.board)
    for row_index in range(ln):
      if self.is_even(row_index) and row_index != 4:
        self.board[row_index] = [colour,0,colour,0,colour,0,colour,0]
      elif not self.is_even(row_index) and row_index != 3:
        self.board[row_index] = [0,colour,0,colour,0,colour,0,colour]
      counter += 1
      if counter == 4:
        colour = 1
    # Messy Setup.
    for row_index in range(ln):
      for col_index in range(ln):
        if self.board[row_index][col_index] != 0:
          colour = self.board[row_index][col_index]
          if colour == 1: 
            colour = RED
          else: 
            colour = WHITE
          self.board[row_index][col_index] = Piece(row_index, col_index, colour) 


  def draw_board(self, win):
    win.fill(LIGHT)
    for row in range(ROWS):
      for col in range(row % 2, COLS, 2):
        p.draw.rect(win, DARK, (row*BOX_SIZE, col*BOX_SIZE, BOX_SIZE, BOX_SIZE))


  def draw_all(self, win):
    # Draw board.
    self.draw_board(win)
    # Draw pieces on board.
    for row in range(ROWS):
      for col in range(COLS):
        piece = self.board[row][col]
        if piece != 0:
          piece.draw(win)


  def get_valid_moves(self, piece, direction):
    # End function if non player piece.
    try:
      # Get row, column (indicies) from tuple object piece.
      row, column = piece.row, piece.column
    except AttributeError:
      print("No valid moves for an empty space!")
      return
    # Potential next moves list.
    next_move_list = []
    # Next row - dependent on player.
    next_row = row + direction
    next_next_row = row + direction + direction
    # List to hold columns on either side.
    left_right = [column-1,column+1]
    # Loop through left right options.
    for next_col in left_right:
      if next_col in range(8):
        next_space = self.whats_in_the_box(next_row, next_col)
        if isinstance(next_space, Piece):
          # Split conditional - case next_space=0, no int attribute colour.
          if next_space.colour != piece.colour:
          # Assign next next column indicies.
            next_next_col = None
            if next_col == column - 1:
              next_next_col = column - 2
            else:
              next_next_col = column + 2
            if self.whats_in_the_box(next_next_row,next_next_col) == 0:
              next_move_list.append((next_next_row, next_next_col))
    # If no forced capture moves yet.
    if not next_move_list:
      # Case: Empty square.
      for next_col in left_right:
        if next_col in range(8):
          # Check state of potential next square.
          if self.whats_in_the_box(next_row, next_col) == 0:
            next_move_list.append((next_row, next_col))
    return next_move_list

  
  def can_capture(self, piece, direction):
    row, col = piece.row, piece.column
    next_row = row + direction
    next_next_row = row + 2 * direction
    for next_col in [col - 1, col + 1]:
      if next_col in range(8):
        next_space = self.whats_in_the_box(next_row, next_col)
        # Is enemy held space.
        if isinstance(next_space, Piece):
          if next_space.colour != piece.colour:
            next_next_col = None
            if next_col == col - 1:
              next_next_col = next_col - 1
            else:
              next_next_col = next_col + 1
            if self.whats_in_the_box(next_next_row, next_next_col) == 0:
              return (next_next_row, next_next_col)
    return False



  def move_piece(self, Piece, new_row, new_col):
    # Temp. remove (parameter) Piece.
    row, col = Piece.row, Piece.column
    self.remove_piece(row,col)
    Piece.move_piece(new_row, new_col)
    self.board[new_row][new_col] = Piece
    # King update.
    if (new_row == 0 or new_row == ROWS - 1) and Piece.king != True:
      Piece.set_king()
      if Piece.colour == RED:
        self.red_king_count += 1
      else:
        self.white_king_count += 1
    # Remove opponent piece:
    if new_row in [row+2, row-2]:
      x = (row+new_row)//2
      y = (col+new_col)//2
      if self.whats_in_the_box(x,y).colour == RED:
          self.red_remaining -= 1
      else:
          self.white_remaining -=1
      self.remove_piece(x,y)
      # Return True if piece is taken.
      return True
    # Else False
    return False


  def whats_in_the_box(self, row, column):
    return self.board[row][column]

  def remove_piece(self, row, col):
    self.board[row][col] = 0

  def has_won(self):
    if red_remaining == 0:
      print("WHITE HAS WON!")
    elif white_remaining == 0:
      print("RED HAS WON")

  def is_even(self,num):
    return (num % 2) == 0 

  def print_board(self):
    print()
    for row in self.board:
      print(row)
    print()

In [35]:
test_gb = GameBoard()
test_gb.setup_board()
test_gb.print_board()

test_gb.can_capture(test_gb.board[2][0], test_gb.board[2][0].direction)


[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, -, 0, -, 0, -]



False

In [41]:
class GameManager:

    def __init__(self, win):
        self.win = win
        self.__init()

    # Hidden method to force reset_game() call.
    def __init(self):
        self.selected_piece = None
        self.gameboard = GameBoard()
        self.valid_moves = []
        self.turn = RED

    def reset_game(self):
        self.__init()

    def update(self):
        self.gameboard.draw_all(self.win)
        self.draw_valid_moves(self.valid_moves)
        self.scorepanel()
        p.display.update()
        p.display.flip()
        
    def scorepanel(self):
        # Panel shape.
        p.draw.rect(self.win, WHITE, (WIDTH,0,98,HEIGHT))
        p.draw.rect(self.win, BLACK, (WIDTH,0,98,HEIGHT), 5)
        # Turn Marker.
            # Text:
        smallfont = p.font.SysFont('Corbel',23) 
        text = smallfont.render('TURN' , True , BLACK)
            # Circle:
        circle_x, circle_y = WIDTH + 50, 50
        if self.turn == RED:
            p.draw.circle(self.win, RED, (circle_x, circle_y), 20)
        else: 
            p.draw.circle(self.win, BLACK, (circle_x, circle_y), 20, 2)
        self.win.blit(text, (circle_x-20.5,circle_y-38))
        # Reset button
        

    def select_piece(self, row, col):
        # Allow for continous selection of pieces.
        if self.selected_piece:
            b = self.move_piece(row, col)
            if not b:
                # Reset selection.
                self.selected_piece = None
                self.select_piece(row,col)
        piece = self.gameboard.whats_in_the_box(row, col)
        if piece.colour == self.turn:
            self.selected_piece = piece
            self.valid_moves = self.gameboard.get_valid_moves(piece, piece.direction)
            return True
        return False
            

    def move_piece(self, row, col):
        piece = self.gameboard.whats_in_the_box(row, col)
        # If selected, piece is not a piece, (row,col) is valid move then move.
        if self.selected_piece and not isinstance(piece, Piece) and (row, col) in self.valid_moves:
            self.gameboard.move_piece(self.selected_piece, row, col)
            # If move is successful switch current turn.
            self.turn_switch()
        else:
            return False
        return True


    def check_capture_moves(self, curr_row, moves):
        default = False
        print(moves)
        for move in moves:
            if move[0] > curr_row + 1:
                default = move
                return default
    
    def turn_switch(self):
        self.valid_moves = []
        if self.turn == RED:
            self.turn = WHITE
        else:
            self.turn = RED


    def draw_valid_moves(self, valid_moves):
        for pos_move in valid_moves:
            row, col = pos_move
            circle_x = col*BOX_SIZE + BOX_SIZE//2
            circle_y = row*BOX_SIZE + BOX_SIZE//2
            colour = None
            if self.turn == RED:
                colour = SALMON
            else:
                colour = SILVER
            p.draw.circle(self.win, colour, (circle_x, circle_y), 10)
            # p.draw.rect(self.win, BLUE, (col*BOX_SIZE, row*BOX_SIZE, BOX_SIZE, BOX_SIZE), width=3)


In [43]:
def get_mouse_pos(pos):
    x, y = pos
    row = y // BOX_SIZE
    col = x // BOX_SIZE
    return row, col


p.display.init()
p.font.init()
SCREENSIZE = (WIDTH+100, HEIGHT)
WIN = p.display.set_mode(SCREENSIZE)
p.display.set_caption("DRAUGHTS")
p.mouse.set_cursor(*p.cursors.tri_left)
FPS = 30

def main():
    run = True
    clock = p.time.Clock()
    gm = GameManager(WIN)
    
    
    while run:
        # Maintain constant frames/second.
        clock.tick(FPS)
        # Look for events during run.
        valid_moves = []
        for event in p.event.get():
            if event.type == p.QUIT:
                run = False
            
            if event.type == p.MOUSEBUTTONDOWN:
                pos = p.mouse.get_pos()
                row, col = get_mouse_pos(pos)
                try:
                    gm.select_piece(row,col)
                except:
                    pass
                # Reset Game.
                if WIDTH + 50 <= pos[0] <= WIDTH + 100 and HEIGHT - 50 <= pos[1]:
                    gm.reset_game()

            if event.type == p.KEYDOWN and event.key == p.K_RETURN:
                print(gm.gameboard.red_remaining)
#                 gm.reset_game()
                
        
        gm.update()
        
    p.display.quit()

main()

10


$$$$

In [10]:
# def get_mouse_pos(pos):
#     x, y = pos
#     row = y // BOX_SIZE
#     col = x // BOX_SIZE
#     return row, col

# p.display.init()
# SCREENSIZE = (WIDTH, HEIGHT)
# WIN = p.display.set_mode(SCREENSIZE)
# p.display.set_caption("DRAUGHTS")
# p.mouse.set_cursor(*p.cursors.tri_left)
# FPS = 30

# def main():
#     run = True
#     clock = p.time.Clock()
#     gm = GameManager(WIN)
#     white_piece = gm.gameboard.whats_in_the_box(2,0)
#     # gm.gameboard.move_piece(white_piece,4,2)

#     while run:
#         # Maintain constant frames/second.
#         clock.tick(FPS)
#         # Look for events during run.
#         valid_moves = []
#         for event in p.event.get():
#             if event.type == p.QUIT:
#                 run = False
            
#             if event.type == p.MOUSEBUTTONDOWN:
#                 pos = p.mouse.get_pos()
#                 row, col = get_mouse_pos(pos)
#                 try:
#                     gm.select_piece(row,col)
#                 except:
#                     pass

# #             if event.type == p.K_SPACE:
# #                 print("Red remaining: {}\nWhite Remaining: {}".format(gm.gameboard.red_remaining, gm.gameboard.white_remaining))
                
        
#         gm.update()
        
#     p.display.quit()

# main()

In [11]:
# Test Cell:
gb = GameBoard()
gb.setup_board()
gb.print_board()

stationary_piece = gb.whats_in_the_box(2,0)
# gb.move_piece(stationary_piece, 3,1)

attack_piece = gb.whats_in_the_box(2,2)
valid_moves = gb.get_valid_moves(attack_piece, attack_piece.direction)
print(valid_moves)



gb.print_board()


[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, -, 0, -, 0, -]

[(3, 1), (3, 3)]

[+, 0, +, 0, +, 0, +, 0]
[0, +, 0, +, 0, +, 0, +]
[+, 0, +, 0, +, 0, +, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, -, 0, -, 0, -, 0, -]
[-, 0, -, 0, -, 0, -, 0]
[0, -, 0, -, 0, -, 0, -]

